In [1]:
import cdsapi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import matplotlib as mpl
import glob
import os
from matplotlib import cm
import matplotlib.dates as mdates
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import mat73
from pytides2.tide import Tide
import pytides2.constituent  as cons
import pytides2.astro as astro
from pyproj import Proj
import matplotlib.pylab as pl
from scipy import signal
from os.path import dirname, join as pjoin
import scipy.io as sio
from svgpathtools import svg2paths
from svgpath2mpl import parse_path
from netCDF4 import Dataset, num2date
from pytz import timezone
import pytz
import missingno as msno
import re 
import mat73
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


# Personalizar graficas
plt.style.use("default")
mpl.rcParams['xtick.labelsize']=12
mpl.rcParams['ytick.labelsize']=12
mpl.rcParams['font.sans-serif'] = "Century Gothic"
mpl.rcParams['font.family'] = "Century Gothic"
mpl.rcParams['font.weight'] = 'normal'
def fig_params(ax):
    ax.grid()    
    ax.spines['left'].set_color('gray')
    ax.spines['right'].set_color('gray')
    ax.spines['bottom'].set_color('gray')
    ax.spines['top'].set_color('gray')
    ax.tick_params('y', colors='tab:gray')
    ax.tick_params('x', colors='tab:gray')
    for label in ax.get_yticklabels():
        label.set_color('k')
        label.set_size(13)
    for label in ax.get_xticklabels():
        label.set_color('k')
        label.set_size(13)


import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [8]:
def download_er5_waves(year_:str, month_:str):
    """ Función para descargar parametros del oleaje de la base de datos ERA5

    Parameters
    ----------
    year_ : str
        Año del cual se descargan datos
    month_ : str
        Mes de a descargar datos
    """
    c = cdsapi.Client()
    c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            'mean_wave_direction', 'peak_wave_period', 'significant_height_of_combined_wind_waves_and_swell',
        ],
        'year': [ f'{year_}',
        ],
        'month': [
        f'{month_}',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': 
            '00:00/ 06:00/ 12:00/ 18:00'
        ,
        'area':[
            '12.875','278','12.25','278.625',
        ],
        'grid':[
            '0.125','0.125',
        ],
        'step' :"0"
    },
    f'G:\My Drive\TESIS_MAESTRIA_PAULAE\MODELO_NUMERICO\BASES_DE_DATOS\CLIMATOLOGIA_ERA5\Data_Waves\ERA5_wave_{year_}_{month_}.nc')

In [11]:
def download_er5_wind(year_:str, month_:str):
    """ Función para descargar las componentes del viento de la base de datos ERA5

    Parameters
    ----------
    year_ : str
        Año del cual se descargan datos
    month_ : str
        Mes de a descargar datos
    """
    c = cdsapi.Client()
    c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind',
        ],
        'year': [
             f'{year_}',
        ],
        'month': [
            
            f'{month_}', 
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00',
            '06:00',
            '12:00',
            '18:00',
        ],
        'area':[
            '12.875','278','12.25','278.625',
        ],
        'grid':[
            '0.125','0.125',
        ],
    },
    f'G:\My Drive\TESIS_MAESTRIA_PAULAE\MODELO_NUMERICO\BASES_DE_DATOS\CLIMATOLOGIA_ERA5\Data_Wind\ERA5_wind_{year_}_{month_}.nc')

In [4]:
meses_numeros = [str(month).zfill(2) for month in range(1, 13)]
print(meses_numeros)
años_ = [str(year) for year in range(2012,2023)]
print(años_)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [10]:
# Descargar climatologicos de olas por año y mes

for year in años_:
    for month in meses_numeros:
        download_er5_waves(year_ = year, month_= month)

2023-05-10 11:01:46,231 INFO Welcome to the CDS
2023-05-10 11:01:46,231 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-05-10 11:01:46,591 INFO Request is completed
2023-05-10 11:01:46,591 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data7/adaptor.mars.internal-1683656307.015467-21078-14-cdf18c69-6068-4fd1-8672-56f0f9e6db05.nc to G:\My Drive\TESIS_MAESTRIA_PAULAE\MODELO_NUMERICO\BASES_DE_DATOS\CLIMATOLOGIA_ERA5\Data_Waves_2\ERA5_wave_2012_01.nc (28.2K)
2023-05-10 11:01:47,808 INFO Download rate 23.2K/s  
2023-05-10 11:01:48,191 INFO Welcome to the CDS
2023-05-10 11:01:48,191 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-05-10 11:01:48,411 INFO Request is completed
2023-05-10 11:01:48,414 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data4/adaptor.mars.internal-1683656323.54636

In [12]:
# Descargar climatologicos de vientos por año y mes 

for year in años_:
    for month in meses_numeros:
        download_er5_wind(year_ = year, month_= month)

2023-05-10 11:05:24,890 INFO Welcome to the CDS
2023-05-10 11:05:24,890 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-05-10 11:05:25,881 INFO Request is completed
2023-05-10 11:05:25,886 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data1/adaptor.mars.internal-1683658898.6037135-22824-17-c911fdb9-33d5-4eaf-b5e4-2708ee5493cd.nc to G:\My Drive\TESIS_MAESTRIA_PAULAE\MODELO_NUMERICO\BASES_DE_DATOS\CLIMATOLOGIA_ERA5\Data_Wind_2\ERA5_wind_2012_01.nc (19.3K)
2023-05-10 11:05:27,124 INFO Download rate 15.6K/s  
2023-05-10 11:05:27,521 INFO Welcome to the CDS
2023-05-10 11:05:27,521 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-05-10 11:05:27,882 INFO Request is completed
2023-05-10 11:05:27,888 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data4/adaptor.mars.internal-1683658913